In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from ResNet import ResNet

In [2]:
transform=transforms.Compose([
    transforms.Resize([96,96]),
    transforms.ToTensor()
])
train_set=torchvision.datasets.CIFAR10(root="../data",train=True,transform=transform,download=False)
test_set=torchvision.datasets.CIFAR10(root="../data",train=False,transform=transform,download=False)

In [3]:
# img,label=train_set[0]
# img.shape,label

In [4]:
batch_size=256
lr=0.1
epoch=20
writer=SummaryWriter("logs")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainLoader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)
testLoader=DataLoader(test_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)

In [5]:
model=ResNet().to(device)

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
model.apply(init_weights)

loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9)

In [6]:
def train(trainLoader,model,loss_fn,optimizer):
    print(f"training on:{device}")
    # model=model.to(device)
    size=len(trainLoader.dataset)
    
    for batch,(X,y) in enumerate(trainLoader):
        X=X.to(device)
        y=y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%10==0:
            loss,current=loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [7]:
max_correct=0
def test(testLoader, model, loss_fn,max_correct):
    # model = model.to(device)
    size = len(testLoader.dataset)
    num_batches = len(testLoader)
    # test_loss, correct = 0, 0
    acc=0
    total=0
    with torch.no_grad():
        for X, y in testLoader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            _,pred=torch.max(pred.data,1)
            total+=y.size(0)
            acc+=(pred==y).sum()
            

    # test_loss /= num_batches

    # correct /= size
    
    # if correct>max_correct:
    #     torch.save(model.state_dict(),'LeNet.params')
    #     max_correct=correct

    # print(f"test error:\n accuracy:{(100 * correct):>0.1f}%,avg loss: {test_loss:>8f} \n")
    
    print(f"test: acc {100 * acc / total}")
    return acc/total
    # return max_correct,correct

In [8]:
for t in range(epoch):
    print(f"epoch {t+1} \n----------------------------------------------")
    train(trainLoader,model,loss,optimizer)
    acc=test(testLoader,model,loss,max_correct)
    writer.add_scalar("acc_ResNet vs epoch",acc,t)
    
writer.close()

epoch 1 
----------------------------------------------
training on:cuda
loss:2.812350 [    0/50000]
loss:2.872330 [ 2560/50000]
loss:2.262649 [ 5120/50000]
loss:2.125688 [ 7680/50000]
loss:2.286537 [10240/50000]
loss:2.044193 [12800/50000]
loss:1.958902 [15360/50000]
loss:1.995030 [17920/50000]
loss:1.866028 [20480/50000]
loss:1.842914 [23040/50000]
loss:1.908438 [25600/50000]
loss:1.835117 [28160/50000]
loss:1.630613 [30720/50000]
loss:1.744376 [33280/50000]
loss:1.574750 [35840/50000]
loss:1.579237 [38400/50000]
loss:1.513280 [40960/50000]
loss:1.568063 [43520/50000]
loss:1.508846 [46080/50000]
loss:1.492773 [48640/50000]
test: acc 44.47999954223633
epoch 2 
----------------------------------------------
training on:cuda
loss:1.570405 [    0/50000]
loss:1.518711 [ 2560/50000]
loss:1.392497 [ 5120/50000]
loss:1.347169 [ 7680/50000]
loss:1.589877 [10240/50000]
loss:1.235840 [12800/50000]
loss:1.451742 [15360/50000]
loss:1.435752 [17920/50000]
loss:1.414106 [20480/50000]
loss:1.286387 

In [9]:
writer.add_scalar("acc_ResNet vs epoch",0,0)
writer.close()